# Feature Engineering

Based on the conlustions from EDA, in this note book, I will conduct feature engineering by combining and normalizing the raw data files to produce one file(named data_use.csv) for training and validation 

In [1]:
import numpy as np
import pandas as pd
from utils import *
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
from sklearn import preprocessing

In [2]:
## getting main training data files (check DA_utils.py for detailed code)
raw_train, raw_store, raw_oil, raw_holiday = get_raw_all()

## Combining the raw_ dataframes and leave the columns we need

In [10]:
use1 = raw_train.merge(raw_store, on  = 'store_nbr').dropna()
use2 = use1.merge(raw_oil, on = 'date', how = 'left').fillna(method = 'bfill')
use2.rename(columns = {'dcoilwtico':'oil_price'}, inplace = True)

In [11]:
use2.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,oil_price
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14


In [12]:
raw_holiday.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [17]:
## Noted that from EDA result,  
## I decided to consider only two holidays in late June as effective holidays
holi_use = raw_holiday.loc[(raw_holiday['locale_name'].isin([
                        'Guaranda', 'Latacunga', 'Machala'])) & 
                          (raw_holiday['description'].isin([
                        'Cantonizacion de Guaranda', 'Cantonizacion de Latacunga',
                              'Fundacion de Machala	']))][['date']]
holi_use['holiday'] = 1

In [30]:
use3 = use2.merge(holi_use,on = 'date',how = 'left').fillna(0)
use3.drop(columns = 'id',inplace = True)
use3.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,oil_price,holiday
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,0.0
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,0.0
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,0.0
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,0.0
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,0.0


## handaling categorical features
The use3 dataframe above is the combintion I need for the modeling. However, I need to deal with the categorical features especailly for regression models and NNs. The method I choose is the "one hot encoding"

In [31]:
use4 = pd.get_dummies(data=use3, columns = ['type','family','city','state','cluster'])

In [32]:
use4.shape

(3000888, 99)

In [33]:
use4.head()

,date,store_nbr,sales,onpromotion,oil_price,holiday,type_A,type_B,type_C,type_D,...,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17
0,2013-01-01,1,0.0,0,93.14,0.0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,2013-01-01,1,0.0,0,93.14,0.0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,2013-01-01,1,0.0,0,93.14,0.0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,2013-01-01,1,0.0,0,93.14,0.0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,2013-01-01,1,0.0,0,93.14,0.0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [34]:
## saving use3 and use4 as using data
# use3.to_csv('use_data.csv')
# use4.to_csv('use_data_dummy.csv')